In [1]:
'''
#Recoginse faces using some classificaiton algorithm -like Logistic KNN SVM etc
1.Read a video stream  using Opencv
2. extract face out of it 
3.Load the training data(numpy arrays of all the persons)
   #x-values are stored in the numpy arrays
   #y-values we need to assign for each person

4.use knn to find the prediction of face(int)
5.map the prediction id to name of the user
6.Display the predictions on the screen -bounding box and name

'''
import matplotlib.pyplot as plt
import cv2
import numpy as np
import os

#######################KNN CODE###################
def distance(v1,v2):
    #Euclidean Distance
    return np.sqrt(((v1-v2)**2).sum())

def knn(train,test,k=5):
    dist=[]

    for i in range(train.shape[0]):
        ix=train[i,:-1]
        iy=train[i,:-1]
        #Compute the distance from test point
        d=distance(test,ix)
        dist.append([d,iy])
    #Sort based on distance and get top k

    dk=sorted(dist,key=lambda x: x[0])[:k]
    #retrieve only the labels
    labels=np.array(dk)[:,-1]
    #Get frequencies of each label
    output=np.unique(labels,return_counts=True)
    index=np.argmax(output[1])
    return output[0][index]
 
 #########################

#init camera
cap=cv2.VideoCapture(0)

face_cascade=cv2.CascadeClassifier("haarcascade_frontalface_alt.xml")
skip=0
dataset_path='./data/'
face_data=[]
labels=[]

class_id=0 #labels for the given file
names={} #Mapping btw id-name

#Data preperation
for fx in os.listdir(dataset_path):
    if fx.endWith('.npy'):
        #Create a mapping btw class_id and name of data file ex sarty.npy so we need only sarty 
        names[class_id]=fx[:-4]

        data_item=np.load(dataset_path+fx)
        face_data.append(data_item)
        #Create Label for the class
        target=class_id*np.ones((data_item.shape[0],))
        class_id+=1
        labels.append(target)

face_dataset=np.concatenate(face_data,axis=0)
face_labels=np.concatenate(labels,axis=0)

trainset=np.concatenate((face_dataset,face_labels),axis=1)

#Testing

while True:
    ret,frame=cap.read()
    if ret==False:
        continue

    faces=face_cascade.detectMultiScale(frame,1.3,5)

    for face in faces:
        x,y,w,h=face
        #Get the face Region of Interest
        offset=10
        face_section=frame[y-offset:y+h+offset,x-offset:x+w+offset]
        face_section=cv2.resize(face_section,(100,100))
        #Predicted label(out)
        out=knn(trainset,face_section.flatten())
        #Display on the screen the name and rectangle around it
        pred_name=names[int(out)]
        cv2.putText(frame,pred_name,(x,y-10),cv2.FONT_HERSHEY_SIMPLEX,1,(255,0,0),2,cv2.LINE_AA)
        cv2.rectangle(frame,(x,y),(x+w,y+h),(0,255,255),2)

    cv2.show("Faces",frame)


    key=cv2.waitKey(1) & 0xFF
    if key==ord('q'):
        break

cap.release()
cv2.destroyAllWindows()







FileNotFoundError: [Errno 2] No such file or directory: './data/'